In [1]:
import pandas as pd
import numpy as np

In [3]:
# 분석할 데이터 load -> auto-mpg.csv 파일을 가져옴
df = pd.read_csv('dataset/auto-mpg.csv', header=None)

df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name']

df.info()
df.horsepower.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB


0    130.0
1    165.0
2    150.0
3    150.0
4    140.0
Name: horsepower, dtype: object

In [7]:
# 데이터 전처리
# horsepower를 구간으로 나누어 분석, 데이터 타입 object -> float
# df['horsepower'] = df['horsepower'].astype('float')
# ? 를 Nan처리 후 삭제
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)

df['horsepower'] = df['horsepower'].astype('float')

In [11]:
# horsepower를 구간으로 나누어 분석 -> 구간을 np.histogram
count, bin_value = np.histogram(df['horsepower'], bins=3)
print(count, bin_value)

# 구간의 이름 부여
bins_names = ['저출력', '보통출력', '고출력']

# 분석하고자 하는 자료에 bin_names를 추가
df['hp_bin'] = pd.cut(x=df['horsepower'], # 처리할 데이터
                     bins=bin_value,      # 경계값 리스트
                      labels = bins_names,# 구간의 이름 리스트
                      include_lowest='True') # 첫 경계값 포함

[257 103  32] [ 46.         107.33333333 168.66666667 230.        ]


In [15]:
df[['horsepower', 'hp_bin']].head(20)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [32]:
# 분류형 자료를 더미변수로 전환
h_dummy = pd.get_dummies(df['hp_bin'])
h_dummy
df = pd.concat([df, h_dummy], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3136 entries, 0 to 397
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   acceleration  392 non-null    float64 
 1   cylinders     392 non-null    float64 
 2   displacement  392 non-null    float64 
 3   horsepower    392 non-null    float64 
 4   hp_bin        392 non-null    category
 5   model year    392 non-null    float64 
 6   mpg           392 non-null    float64 
 7   name          392 non-null    object  
 8   origin        392 non-null    float64 
 9   weight        392 non-null    float64 
 10  고출력           2744 non-null   float64 
 11  보통출력          2744 non-null   float64 
 12  저출력           2744 non-null   float64 
dtypes: category(1), float64(11), object(1)
memory usage: 321.7+ KB


In [28]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.alive.unique()
alive_yes = pd.get_dummies(titanic['alive'], prefix='alive', drop_first=True)

In [29]:
df1 = pd.concat([titanic, alive_yes], axis=1)
df1[['alive', 'alive_yes']].head()

,alive,alive_yes
0,no,0
1,yes,1
2,yes,1
3,yes,1
4,no,0


In [41]:
# one_hot_encoding 으로 더미 변수 생성
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 생성
label_encoder = preprocessing.LabelEncoder()   # label_encoder 생성
onehot_encoder = preprocessing.OneHotEncoder() # onehot_encoder 생성

# label_encoder 문자열 함수를 숫자형 범주로 변환
onehot_label = label_encoder.fit_transform(df['hp_bin'])
len(onehot_label)
print(onehot_label.ndim, onehot_label.dtype, onehot_label.size)

# 2차원 행렬로 변환
onehot_reshape = onehot_label.reshape(len(onehot_label),1)
print(onehot_reshape.ndim, onehot_reshape.dtype, onehot_reshape.size)
onehot_reshape[:10, :]

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshape)
print(onehot_fitted)
print(type(onehot_fitted))

1 int32 3136
2 int32 3136
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
  (15, 2)	1.0
  (16, 2)	1.0
  (17, 2)	1.0
  (18, 2)	1.0
  (19, 2)	1.0
  (20, 2)	1.0
  (21, 2)	1.0
  (22, 2)	1.0
  (23, 1)	1.0
  (24, 2)	1.0
  :	:
  (3111, 3)	1.0
  (3112, 3)	1.0
  (3113, 3)	1.0
  (3114, 3)	1.0
  (3115, 3)	1.0
  (3116, 3)	1.0
  (3117, 3)	1.0
  (3118, 3)	1.0
  (3119, 3)	1.0
  (3120, 3)	1.0
  (3121, 3)	1.0
  (3122, 3)	1.0
  (3123, 3)	1.0
  (3124, 3)	1.0
  (3125, 3)	1.0
  (3126, 3)	1.0
  (3127, 3)	1.0
  (3128, 3)	1.0
  (3129, 3)	1.0
  (3130, 3)	1.0
  (3131, 3)	1.0
  (3132, 3)	1.0
  (3133, 3)	1.0
  (3134, 3)	1.0
  (3135, 3)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


In [43]:
# 시계열 데이터 처리
df = pd.read_csv('dataset/stock-data.csv')
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


,Date,Close,Start,High,Low,Volume
0,2018-07-02,10100,10850,10900,10000,137977
1,2018-06-29,10700,10550,10900,9990,170253
2,2018-06-28,10400,10900,10950,10150,155769
3,2018-06-27,10900,10800,11050,10500,133548
4,2018-06-26,10800,10900,11000,10700,63039
5,2018-06-25,11150,11400,11450,11000,55519
6,2018-06-22,11300,11250,11450,10750,134805
7,2018-06-21,11200,11350,11750,11200,133002
8,2018-06-20,11550,11200,11600,10900,308596
9,2018-06-19,11300,11850,11950,11300,180656


In [45]:
df['new_Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB


In [48]:
# new_Date를 인덱스 컬럼으로 설정, Date 컬럼을 삭제
df.drop('Date', axis=1, inplace=True)
df.head(10)

,Close,Start,High,Low,Volume
new_Date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039
2018-06-25,11150,11400,11450,11000,55519
2018-06-22,11300,11250,11450,10750,134805
2018-06-21,11200,11350,11750,11200,133002
2018-06-20,11550,11200,11600,10900,308596


In [49]:
ts_ms = pd.date_range(start='2019-01-01',    # 날짜 범위의 시작
                   end=None,                 # 날짜 범위의 끝
                   periods=6,                # 생성할 Timestamp의 개수
                   freq='MS',                # 시간 간격 (MS: 월의 시작일)
                   tz='Asia/Seoul')          # 시간대(timezone)
print(ts_ms)
print('\n')

# 월 간격, 월의 마지막 날 기준
ts_me = pd.date_range('2019-01-01', periods=6, 
                   freq='M',              # 시간 간격 (M: 월의 마지막 날)
                   tz='Asia/Seoul')       # 시간대(timezone)
print(ts_me)
print('\n')

# 분기(3개월) 간격, 월의 마지막 날 기준
ts_3m = pd.date_range('2019-01-01', periods=6, 
                   freq='3M',             # 시간 간격 (3M: 3개월)
                   tz='Asia/Seoul')       # 시간대(timezone)
print(ts_3m)
# Period 배열 만들기 - 1개월 길이
pr_m = pd.period_range(start='2019-01-01',     # 날짜 범위의 시작
                   end=None,                   # 날짜 범위의 끝
                   periods=3,                  # 생성할 Period 개수
                   freq='M')                   # 기간의 길이 (M: 월)
print(pr_m)
print('\n')

# Period 배열 만들기 - 1시간 길이
pr_h = pd.period_range(start='2019-01-01',     # 날짜 범위의 시작
                   end=None,                   # 날짜 범위의 끝
                   periods=3,                  # 생성할 Period 개수
                   freq='H')                   # 기간의 길이 (H: 시간)
print(pr_h)
print('\n')

# Period 배열 만들기 - 2시간 길이
pr_2h = pd.period_range(start='2019-01-01',    # 날짜 범위의 시작
                   end=None,                   # 날짜 범위의 끝
                   periods=3,                  # 생성할 Period 개수
                   freq='2H')                  # 기간의 길이 (H: 시간)
print(pr_2h)


DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')
PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]')


PeriodIndex(['2019-01-01 00:00', '2019-01-01 01:00', '2019-01-01 02:00'], dtype='perio

In [52]:
# 날짜 데이터 분리 년- 월- 일
df = pd.read_csv('dataset/stock-data.csv')

df['new_Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
df

,Date,Close,Start,High,Low,Volume,new_Date,Year,Month,Day
0,2018-07-02,10100,10850,10900,10000,137977,2018-07-02,2018,7,2
1,2018-06-29,10700,10550,10900,9990,170253,2018-06-29,2018,6,29
2,2018-06-28,10400,10900,10950,10150,155769,2018-06-28,2018,6,28
3,2018-06-27,10900,10800,11050,10500,133548,2018-06-27,2018,6,27
4,2018-06-26,10800,10900,11000,10700,63039,2018-06-26,2018,6,26
5,2018-06-25,11150,11400,11450,11000,55519,2018-06-25,2018,6,25
6,2018-06-22,11300,11250,11450,10750,134805,2018-06-22,2018,6,22
7,2018-06-21,11200,11350,11750,11200,133002,2018-06-21,2018,6,21
8,2018-06-20,11550,11200,11600,10900,308596,2018-06-20,2018,6,20
9,2018-06-19,11300,11850,11950,11300,180656,2018-06-19,2018,6,19
